In [44]:
import pandas as pd 
import os
import xmltodict
import numpy as np
import xml.etree.ElementTree as etree
from collections import OrderedDict
pd.options.display.max_rows  = 500
pd.options.display.max_colwidth = -1
print("successfully imported")

successfully imported


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [45]:
tree = etree.parse("/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml")
root = tree.getroot()

def flatten_dict(d):
    def items():
        for key, value in d.items():
            if isinstance(value, dict):
                for subkey, subvalue in flatten_dict(value).items():
                    yield key + "." + subkey, subvalue
            else:
                yield key, value
    return OrderedDict(items())

def flatten_dict_detailed(d):
    def items():
        for key, value in d.items():
            if isinstance(value, dict):
                for i, (subkey, subvalue) in enumerate(flatten_dict_detailed(value).items()):
                    yield key + f"{i}" + "." + subkey, subvalue
            elif isinstance(value, list):
                for i, val in enumerate(value):
                    if isinstance(val, str):
                        yield key + f'{i}', val
                    if isinstance(val, dict):
                        for subkey, subvalue in flatten_dict_detailed(val).items():
                            yield key + f'{i}' + "." + subkey, subvalue
                    if isinstance(val, list):
                        yield flatten_dict_detailed(val)
            else:
                yield key, value
    
    return OrderedDict(items())

# Convert XML to dict
#TODO: create better CCD ingestion 
with open('/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml', 'rb') as f:
    xml_content = xmltodict.parse(f)

# Flatten dict
flattened_xml = flatten_dict(xml_content)
agg = []

# Print in desired format
for k,v in flattened_xml.items():
    agg.append((k,v))
    
flattened_df = pd.DataFrame(agg, columns = ['tags', 'value'])


In [46]:
def CCDAIngest(filepath):
    with open(filepath, 'rb') as f:
        xml_content = xmltodict.parse(f)
    flattened_xml = flatten_dict(xml_content)
    agg = []
    for k,v in flattened_xml.items():
        agg.append((k,v))
    flattened_df = pd.DataFrame(agg, columns=['tag', 'value'])
    return flattened_df

def get_components(CCDA, nested_indexes, key='tag'):
    nested_ccda_df = CCDA.loc[nested_indexes]
    index = nested_ccda_df[nested_ccda_df[key].str.contains('component')].index
    return nested_ccda_df.loc[index[0]].value


In [47]:
df = CCDAIngest("/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml")

In [48]:
#@ = attribute
#. = next node
flattened_df.head(10)

,tags,value
0,ClinicalDocument.@xmlns,urn:hl7-org:v3
1,ClinicalDocument.@xmlns:xsi,http://www.w3.org/2001/XMLSchema-instance
2,ClinicalDocument.@xmlns:voc,urn:hl7-org:v3/voc
3,ClinicalDocument.@xmlns:sdtc,urn:hl7-org:sdtc
4,ClinicalDocument.realmCode.@code,US
5,ClinicalDocument.typeId.@extension,POCD_HD000040
6,ClinicalDocument.typeId.@root,2.16.840.1.113883.1.3
7,ClinicalDocument.templateId,"[{'@root': '2.16.840.1.113883.10.20.22.1.2', '@extension': '2015-08-01'}, {'@root': '2.16.840.1.113883.10.20.22.1.2'}]"
8,ClinicalDocument.id.@extension,TT988
9,ClinicalDocument.id.@root,2.16.840.1.113883.19.5.99999.1


In [49]:
#Find indexes where value is not string
nested_indexes = []
for i, val in enumerate(flattened_df.value):
    if type(val) == list:
        nested_indexes.append(i)

In [50]:
flattened_df.loc[nested_indexes].shape

(9, 2)

In [51]:
#flattened_df.loc[nested_indexes]

In [52]:
#hard coding for now
informant = pd.DataFrame(flattened_df.loc[131].value)
participant = pd.DataFrame(flattened_df.loc[186].value)
components = pd.DataFrame(flattened_df.loc[223].value)

In [53]:
#each row is another component (allergies, advance directives etc.)
components.shape

(15, 1)

In [54]:
#immunizations section
Immunizations = flatten_dict(components.section[5])
Immunizations_df = pd.DataFrame(flatten_dict_detailed(Immunizations).items(), columns = ['tag', 'value'])

In [55]:
df = Immunizations_df[Immunizations_df.tag.str.contains('id[0-9].@root', regex=True)]
df[df.duplicated(subset='value')]

,tag,value
107,entry1.substanceAdministration6.id0.@root,e6f1ba43-c0ed-4b9b-9f12-f435d8ad8f92
127,entry1.substanceAdministration26.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456
138,entry1.substanceAdministration37.performer11.assignedEntity11.representedOrganization0.id0.@root,2.16.840.1.113883.19.5.9999.1394
168,entry2.substanceAdministration6.id0.@root,e6f1ba43-c0ed-4b9b-9f12-f435d8ad8f92
188,entry2.substanceAdministration26.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456
199,entry2.substanceAdministration37.performer11.assignedEntity11.representedOrganization0.id0.@root,2.16.840.1.113883.19.5.9999.1394
233,entry3.substanceAdministration6.id0.@root,e6f1ba43-c0ed-4b9b-9f12-f435d8ad8f92
253,entry3.substanceAdministration26.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456
264,entry3.substanceAdministration37.performer11.assignedEntity11.representedOrganization0.id0.@root,2.16.840.1.113883.19.5.9999.1394
302,entry4.substanceAdministration22.performer0.assignedEntity0.id0.@root,2.16.840.1.113883.19.5.9999.456


In [56]:
def check_duplicates_ids(df, tag_column='tag', value_column='value', regex='id[0-9].@root'):
    new_df = df[df[tag_column].str.contains(regex, regex=True)]
    title = df.loc[df['tag'] == 'title', 'value'].values[0]
    duplicated = new_df[new_df.duplicated(subset=value_column)]
    if not duplicated.empty:
        return (title, "True")
    else:
        return (title, "False")

check_duplicates_ids(Immunizations_df)

('IMMUNIZATIONS', 'True')

In [57]:
duplicated = []
for x in components['section']:
    this_df = pd.DataFrame(flatten_dict_detailed(flatten_dict(x)).items(), columns = ['tag', 'value'])
    duplicated.append(check_duplicates_ids(this_df))
    